In [67]:
%load_ext autoreload
%autoreload 1

%aimport basics
%aimport nn_config
%aimport enc_dec


from basics import *
from nn_config import *
from enc_dec import *
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
from enc_dec import *

In [69]:
xp

<module 'cupy' from '/afs/inf.ed.ac.uk/user/s14/s1444673/anaconda3/envs/mtenv/lib/python3.6/site-packages/cupy/__init__.py'>

In [70]:
xp = cuda.cupy if gpuid >= 0 else np

In [71]:
text_data = pickle.load(open(text_data_dict, "rb"))

In [72]:
text_data.keys()

dict_keys(['train', 'dev', 'test'])

In [73]:
model = SpeechEncoderDecoder(120, vocab_size_en, num_layers_enc, num_layers_dec,
                               hidden_units, gpuid, attn=use_attn)

In [74]:
sp_fil = np.load(os.path.join(speech_dir, "041.001"+speech_extn))

In [75]:
t_sp_fil = xp.expand_dims(sp_fil, 1)

In [76]:
t_sp_fil.shape, t_sp_fil[0].shape, sp_fil[0].shape

((1168, 1, 120), (1, 120), (120,))

In [77]:
np.expand_dims(sp_fil[:5],0).shape

(1, 5, 120)

In [78]:
sp_fil.shape

(1168, 120)

In [79]:
xp.flip(sp_fil, axis=0)

array([[-1.19359016, -1.41158342, -1.60597098, ...,  0.0163932 ,
        -0.10014834, -0.274268  ],
       [-1.43545854, -1.55333805, -1.15664506, ...,  0.14043394,
        -0.02434985, -0.2729277 ],
       [-0.73096627, -1.39472294, -1.21697617, ...,  0.28072414,
         0.15903132,  0.01507408],
       ..., 
       [ 0.02010632,  0.00897999, -0.13785338, ..., -0.46035278,
        -0.4079282 , -0.81319255],
       [-1.9217509 , -2.3823216 , -1.18668556, ...,  0.05905694,
        -0.1516051 , -0.36329484],
       [-5.12446594, -4.33465815, -3.91597891, ...,  0.32548103,
         0.17863652,  0.14788523]], dtype=float32)

In [80]:
t_lstm = L.LSTM(sp_fil.shape[1], 200)

In [81]:
A = t_lstm(sp_fil)

In [82]:
(A.data==t_lstm.h.data).all()

True

In [83]:
t_lstm(sp_fil[:5]).shape

(5, 200)

In [84]:
len(sp_fil), t_lstm.h.shape, len(t_sp_fil)

(1168, (1168, 200), 1168)

In [85]:
model.L1_enc.state_size, sp_fil.shape[0] / 8

(128, 146.0)

In [86]:
# out_states = model.encode_speech_lstm(sp_fil, train=False)

In [88]:
model.L2_rev_enc.state_size

128

In [89]:
L_REV_STATES = model.encode_speech_lstm(xp.flip(sp_fil, axis=0), model.lstm_rev_enc, train=True)

speech (1168, 1, 120)
L0_rev_enc before (1168, 1, 120)
L0_rev_enc out (584, 1, 256)
L1_rev_enc before (584, 1, 256)
L1_rev_enc out (292, 1, 256)
L2_rev_enc before (292, 1, 256)
L2_rev_enc out (146, 1, 256)
L3_rev_enc before (146, 1, 256)
L3_rev_enc out (73, 1, 256)
L4_rev_enc before (73, 1, 256)
L4_rev_enc out (73, 1, 128)


In [90]:
out_states = model.encode_list(sp_fil, train=True)

speech (1168, 1, 120)
L0_enc before (1168, 1, 120)
L0_enc out (584, 1, 256)
L1_enc before (584, 1, 256)
L1_enc out (292, 1, 256)
L2_enc before (292, 1, 256)
L2_enc out (146, 1, 256)
L3_enc before (146, 1, 256)
L3_enc out (73, 1, 256)
L4_enc before (73, 1, 256)
L4_enc out (73, 1, 128)
speech (1168, 1, 120)
L0_rev_enc before (1168, 1, 120)
L0_rev_enc out (584, 1, 256)
L1_rev_enc before (584, 1, 256)
L1_rev_enc out (292, 1, 256)
L2_rev_enc before (292, 1, 256)
L2_rev_enc out (146, 1, 256)
L3_rev_enc before (146, 1, 256)
L3_rev_enc out (73, 1, 256)
L4_rev_enc before (73, 1, 256)
L4_rev_enc out (73, 1, 128)


In [58]:
A = xp.random.rand(4,3).astype(xp.float32)

In [59]:
A

array([[ 0.00864284,  0.36081648,  0.41948977],
       [ 0.9396922 ,  0.91287845,  0.99141073],
       [ 0.33583364,  0.83446676,  0.88580513],
       [ 0.62025827,  0.26510096,  0.9786287 ]], dtype=float32)

In [60]:
xp.flip(A, axis=0)

array([[ 0.62025827,  0.26510096,  0.9786287 ],
       [ 0.33583364,  0.83446676,  0.88580513],
       [ 0.9396922 ,  0.91287845,  0.99141073],
       [ 0.00864284,  0.36081648,  0.41948977]], dtype=float32)

In [ ]:
out_states.shape, model.lstm_enc

In [ ]:
out_states = model.feed_pyramidal_lstm(t_sp_fil[:9], lstm_layer=model.lstm_enc[0], scale=2, train=False)

In [ ]:
out_states.shape

In [ ]:
# out_states = feed_pyramidal_lstm(sp_fil[:10], scale=2, train=False)

In [ ]:
out_states.shape

In [ ]:
def feed_pyramidal_lstm(feat_in, scale, train):
    xp = cuda.cupy if gpuid >= 0 else np

    # create empty array to store the output
    # the output is scaled by the scale factor
    n_out_states = len(feat_in) // scale
    out_dim = t_lstm.state_size
    out_states = xp.empty((0, out_dim*scale), dtype=xp.float32)

    # feed each input from the sequence
    for i in range(0, n_out_states):
        # lateral_states = xp.empty((0,out_dim), dtype=xp.float32)
        lateral_states = t_lstm(xp.expand_dims(feat_in[(i*scale)],0))
        for j in range(1, scale):
            out = t_lstm(xp.expand_dims(feat_in[(i*scale)+j],0))
            print(out.shape)
            print(lateral_states.shape)
            lateral_states = F.concat((lateral_states, out), axis=1)
        # concatenate and append lateral states into out states
        out_states = F.concat((out_states, lateral_states), axis=0)

    return out_states

In [ ]:
feat_in = np.random.rand(10, 40).astype(np.float32)
scale = 2
n_out_states = len(feat_in) // scale
print(n_out_states)

In [ ]:
haha=np.empty(30, dtype=np.float32)

In [ ]:
haha.shape

In [ ]:
for i in range(0, n_out_states):
    for j in range(0, scale):
        print((i*scale)+j)

In [ ]:
for i in range(0, len(feat_in), scale):
    lateral_states = xp.empty((scale, out_dim), dtype=xp.float32)
    for j in range(i, i+scale):
        lateral_states[j-i] = self[lstm_layer](feat_in[j])
    # concatenate and append lateral states into out states
    n_out_states[]

In [ ]:
for i in range(0,10,2):
    for j in range(i, i+2):
        print(j, end=",")
    print("")

In [ ]:
%aimport prepare_dataset

In [ ]:
frame_lengths = prepare_dataset.make_npz("haha", "train")

In [ ]:
len(frame_lengths)

In [ ]:
len([l for l in frame_lengths if l>=400])

In [ ]:
plt.hist(frame_lengths)